lab-spotify-API

In [1]:
import requests

In [2]:
import functions

In [3]:
import pandas as pd
import numpy as np

In [4]:
#!conda install -c conda-forge spotipy -y

In [5]:
# config file for spotify app
import sys

from config import *

In [6]:
#spotify authentication
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias #
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_ID,
                                                           client_secret=Client_Secret))

In [7]:
# TEST: Search for an artist
#results = sp.search(q='Mannarino', type='artist')

# Print the results
#for idx, t in enumerate(results['artists']['items']):
#   print(idx, t['name'])

In [8]:
# test 
#results = sp.search(q="Mannarino", limit=5)

In [9]:
# test
#for idx, track in enumerate(results['tracks']['items']):
#    print(idx, track['name'], " – ", track['artists'][0]['name'])


### Import Data (music)

In [10]:
#import data
data = pd.read_csv('billboard_top_100.csv')

### 1 create function to seach for song:

# 1 create function to seach for song:

from typing import Optional

def search_song(title: str, artist: str) -> Optional[str]:
    '''
    This function uses Spotify's API to find a song ID based on its title and artist.
    inputs:
       title (str): The title of the song.
       artist (str): The artist of the song.
       
    outputs:
       Optional[str]: The ID of the song if it exists, otherwise None.
    '''
    results = sp.search(q=f'track:{title} artist:{artist}', type='track')
    items = results['tracks']['items']

    if len(items) > 0:
        song = items[0]
        song_id=(song['id'])
    else:
        song_id = "Song not found"
    return song_id


In [11]:
# test function
#song_id = functions.search_song('little sister', 'Queens of the stone age', sp)
# song_id


In [12]:
import time

# create function for creating chunks
def split_into_chunks(df:pd.DataFrame, rows_per_chunk:int=25) -> list[pd.DataFrame]:
    '''
    create chunks based on parameters
    input: (df:pd.DataFrame, rows_per_chunk:int=25)
    output: list of dataframes
    '''
    num_chunks = round(len(df['Song_title'])/rows_per_chunk) #calculate number of chunks needed
    chunks = np.array_split(df, num_chunks) #split data into chunks
    return chunks

In [13]:
test_c = functions.split_into_chunks(data, 25)

/Users/alexferrer/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


# 2 create list of ids

def list_of_song_ids(df:pd.DataFrame) -> list:
    '''
    This function takes a DataFrame as input, splits it into chunks of 25 rows each, and processes each chunk separately. 
    For each row in a chunk, it tries to find the song ID using the song title and artist. 
    If successful, it appends the song ID to a list; otherwise, it appends an empty string. 
    After processing each chunk, it waits for 25 seconds before moving to the next chunk. 
    Finally, it returns the list of song IDs.
    '''
    list_of_ids = [] #create empty list
    chunks = split_into_chunks(df, 25) #using defined function to create chunks
    for chunk_df in chunks:
        for index, row in chunk_df.iterrows():
            try:
                song_id = search_song(row['Song_title'], row['Artist'])         
                print(f"Song Title: {row['Song_title']}, Artist: {row['Artist']}, Song ID: {song_id}")
                list_of_ids.append(song_id)
            except:
                print("Song not found")
                list_of_ids.append("")
        print("sleep a bit before getting the next chunk")  
        time.sleep(25)
    return list_of_ids


In [14]:
data

,Song_title,Artist
0,Lovin On Me,Jack Harlow
1,Cruel Summer,Taylor Swift
2,Greedy,Tate McRae
3,Paint The Town Red,Doja Cat
4,I Remember Everything,Zach Bryan Featuring Kacey Musgraves
...,...,...
95,Tourniquet,Zach Bryan
96,Y Lloro,Junior H
97,Murder On The Dancefloor,Sophie Ellis-Bextor
98,Amargura,Karol G


In [15]:
list_of_ids = functions.list_of_song_ids(data, sp)

/Users/alexferrer/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Song Title: Lovin On Me, Artist: Jack Harlow, Song ID: 4xhsWYTOGcal8zt0J161CU
Song Title: Cruel Summer, Artist: Taylor Swift, Song ID: 1BxfuPKGuaTgP7aM0Bbdwr
Song Title: Greedy, Artist: Tate McRae, Song ID: 3rUGC1vUpkDG9CZFHMur1t
Song Title: Paint The Town Red, Artist: Doja Cat, Song ID: 2IGMVunIBsBLtEQyoI1Mu7
Song Title: I Remember Everything, Artist: Zach Bryan Featuring Kacey Musgraves, Song ID: Song not found
Song Title: Snooze, Artist: SZA, Song ID: 4iZ4pt7kvcaH6Yo8UoZ4s2
Song Title: Water, Artist: Tyla, Song ID: 5aIVCx5tnk0ntmdiinnYvw
Song Title: Last Night, Artist: Morgan Wallen, Song ID: 7K3BhSpAxZBznislvUMVtn
Song Title: Fast Car, Artist: Luke Combs, Song ID: 1Lo0QY9cvc8sUB2vnIOxDT
Song Title: Agora Hills, Artist: Doja Cat, Song ID: 7dJYggqjKo71KI9sLzqCs8
Song Title: Thinkin' Bout Me, Artist: Morgan Wallen, Song ID: 0PAcdVzhPO4gq1Iym9ESnK
Song Title: Lose Control, Artist: Teddy Swims, Song ID: 6usohdchdzW9oML7VC4Uhk
Song Title: Is It Over Now? (Taylor's Version) [From The Vaul

In [16]:
len(list_of_ids)

100

In [17]:
data_test=data.copy()

In [18]:
data_test['id']=list_of_ids

In [19]:
data_test

,Song_title,Artist,id
0,Lovin On Me,Jack Harlow,4xhsWYTOGcal8zt0J161CU
1,Cruel Summer,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr
2,Greedy,Tate McRae,3rUGC1vUpkDG9CZFHMur1t
3,Paint The Town Red,Doja Cat,2IGMVunIBsBLtEQyoI1Mu7
4,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,Song not found
...,...,...,...
95,Tourniquet,Zach Bryan,3EvZ03hGAFwGZ2Ebcu86YH
96,Y Lloro,Junior H,6RcAHyC5sAUIbPTkhOQwd8
97,Murder On The Dancefloor,Sophie Ellis-Bextor,4tKGFmENO69tZR9ahgZu48
98,Amargura,Karol G,505v13epFXodT9fVAJ6h8k


In [20]:
# remove missing IDs and reset index
hot_100_with_ids = data_test[data_test['id'] != "Song not found"]
hot_100_with_ids = hot_100_with_ids.reset_index(drop=True)

In [21]:
len(hot_100_with_ids)

84

In [22]:
type(hot_100_with_ids['id'])

pandas.core.series.Series

In [15]:
# test: find name using song id
#track_info = sp.track('0BqQWfhMrkpRAUCbdfdHUC')
#print(track_info['name'])


Little Sister


# 3 Function: Get the audio features of a song
from typing import List, Optional
import pandas as pd

def get_audio_features(df:pd.DataFrame, id_column:str='id') -> Optional[pd.DataFrame]:

    '''
    This function uses Spotify's API to get the audio features of songs based on their IDs.
    inputs:
    df (pd.DataFrame): A DataFrame containing song IDs.
    id_column (str): The name of the column in the DataFrame that contains the song IDs.
    outputs:
    pd.DataFrame: A DataFrame containing the audio features of the songs. If a request fails, the corresponding row will contain NaN values.
    '''
    chunks = split_into_chunks(df, 20) #using defined function to create chunks
    all_audio_features = []
    
    for chunk_df in chunks:
        try:
            audio_features = sp.audio_features(chunk_df[id_column])
            audio_features_df = pd.DataFrame(audio_features)
            print(audio_features_df)
            all_audio_features.append(audio_features_df)
        except Exception as e:
            print(f"An error occurred: {e}")
            all_audio_features.append(pd.DataFrame(np.nan, index=range(len(chunk_df)), columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url']))
        print("sleep a bit before getting the next chunk")  
        time.sleep(25)
        
    merged_features = pd.concat(all_audio_features, ignore_index=True)
    return merged_features

In [24]:
audio_features = functions.get_audio_features(hot_100_with_ids, 'id', sp)

/Users/alexferrer/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


    danceability  energy  key  loudness  mode  speechiness  acousticness  \
0          0.943  0.5580    2    -4.911     1       0.0568        0.0026   
1          0.552  0.7020    9    -5.707     1       0.1570        0.1170   
2          0.750  0.7330    6    -3.180     0       0.0319        0.2560   
3          0.868  0.5380    5    -8.603     1       0.1740        0.2690   
4          0.559  0.5510    5    -7.231     1       0.1320        0.1410   
5          0.673  0.7220    3    -3.495     0       0.0755        0.0856   
6          0.492  0.6750    6    -5.456     1       0.0389        0.4670   
7          0.712  0.6030    8    -5.520     1       0.0262        0.1860   
8          0.750  0.6740    8    -6.128     0       0.0970        0.2280   
9          0.656  0.7570    3    -5.775     0       0.0308        0.4920   
10         0.561  0.6040    9    -4.409     1       0.0337        0.1990   
11         0.596  0.6580    0    -7.346     1       0.0360        0.0504   
12         0

In [26]:
audio_features.shape

(84, 18)

# 6 create function to concatenate id dataframe with audio features

def add_audio_features(df_ids: pd.DataFrame, df_audio_features: pd.DataFrame) -> pd.DataFrame:
   """
   Concatenates two dataframes.

   Inputs:
   df_ids (pd.DataFrame): The first dataframe with ids.
   df_audio_features (pd.DataFrame): The second dataframe with audio features.

   Output:
   pd.DataFrame: The concatenated dataframe.
   """
   return pd.concat([df_ids, df_audio_features], axis=1)

In [35]:
hot_songs_with_features=add_audio_features(hot_100_with_ids, audio_features)

In [28]:
hot_songs_with_features

,Song_title,Artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Lovin On Me,Jack Harlow,4xhsWYTOGcal8zt0J161CU,0.943,0.558,2,-4.911,1,0.0568,0.00260,...,0.0937,0.606,104.983,audio_features,4xhsWYTOGcal8zt0J161CU,spotify:track:4xhsWYTOGcal8zt0J161CU,https://api.spotify.com/v1/tracks/4xhsWYTOGcal...,https://api.spotify.com/v1/audio-analysis/4xhs...,138411,4
1,Cruel Summer,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr,0.552,0.702,9,-5.707,1,0.1570,0.11700,...,0.1050,0.564,169.994,audio_features,1BxfuPKGuaTgP7aM0Bbdwr,spotify:track:1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://api.spotify.com/v1/audio-analysis/1Bxf...,178427,4
2,Greedy,Tate McRae,3rUGC1vUpkDG9CZFHMur1t,0.750,0.733,6,-3.180,0,0.0319,0.25600,...,0.1140,0.844,111.018,audio_features,3rUGC1vUpkDG9CZFHMur1t,spotify:track:3rUGC1vUpkDG9CZFHMur1t,https://api.spotify.com/v1/tracks/3rUGC1vUpkDG...,https://api.spotify.com/v1/audio-analysis/3rUG...,131872,1
3,Paint The Town Red,Doja Cat,2IGMVunIBsBLtEQyoI1Mu7,0.868,0.538,5,-8.603,1,0.1740,0.26900,...,0.0901,0.732,99.968,audio_features,2IGMVunIBsBLtEQyoI1Mu7,spotify:track:2IGMVunIBsBLtEQyoI1Mu7,https://api.spotify.com/v1/tracks/2IGMVunIBsBL...,https://api.spotify.com/v1/audio-analysis/2IGM...,231750,4
4,Snooze,SZA,4iZ4pt7kvcaH6Yo8UoZ4s2,0.559,0.551,5,-7.231,1,0.1320,0.14100,...,0.1100,0.392,143.008,audio_features,4iZ4pt7kvcaH6Yo8UoZ4s2,spotify:track:4iZ4pt7kvcaH6Yo8UoZ4s2,https://api.spotify.com/v1/tracks/4iZ4pt7kvcaH...,https://api.spotify.com/v1/audio-analysis/4iZ4...,201800,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Tourniquet,Zach Bryan,3EvZ03hGAFwGZ2Ebcu86YH,0.593,0.397,6,-8.309,1,0.0329,0.68400,...,0.0982,0.320,76.703,audio_features,3EvZ03hGAFwGZ2Ebcu86YH,spotify:track:3EvZ03hGAFwGZ2Ebcu86YH,https://api.spotify.com/v1/tracks/3EvZ03hGAFwG...,https://api.spotify.com/v1/audio-analysis/3EvZ...,189053,4
80,Y Lloro,Junior H,6RcAHyC5sAUIbPTkhOQwd8,0.728,0.589,7,-7.115,1,0.0376,0.42400,...,0.1660,0.767,77.475,audio_features,6RcAHyC5sAUIbPTkhOQwd8,spotify:track:6RcAHyC5sAUIbPTkhOQwd8,https://api.spotify.com/v1/tracks/6RcAHyC5sAUI...,https://api.spotify.com/v1/audio-analysis/6RcA...,179013,4
81,Murder On The Dancefloor,Sophie Ellis-Bextor,4tKGFmENO69tZR9ahgZu48,0.730,0.849,1,-5.281,0,0.0299,0.00234,...,0.3120,0.887,117.310,audio_features,4tKGFmENO69tZR9ahgZu48,spotify:track:4tKGFmENO69tZR9ahgZu48,https://api.spotify.com/v1/tracks/4tKGFmENO69t...,https://api.spotify.com/v1/audio-analysis/4tKG...,230013,4
82,Amargura,Karol G,505v13epFXodT9fVAJ6h8k,0.920,0.696,6,-3.356,0,0.0742,0.18300,...,0.1490,0.545,106.966,audio_features,505v13epFXodT9fVAJ6h8k,spotify:track:505v13epFXodT9fVAJ6h8k,https://api.spotify.com/v1/tracks/505v13epFXod...,https://api.spotify.com/v1/audio-analysis/505v...,170480,4


In [29]:
hot_songs_with_features.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
mean,0.657548,0.640775,5.273810,-6.390036,0.583333,0.073896,0.236043,0.012013,0.168754,0.472054,123.240345,192963.678571,3.869048
std,0.146135,0.159351,3.654841,2.850934,0.495968,0.063567,0.262566,0.068522,0.104436,0.240502,25.643712,40849.653694,0.432971
min,0.353000,0.091100,0.000000,-17.665000,0.000000,0.024800,0.000219,0.000000,0.023200,0.066200,76.703000,114234.000000,1.000000
25%,0.527750,0.557750,2.000000,-7.373250,0.000000,0.035325,0.043900,0.000000,0.104750,0.256250,106.728750,163791.250000,4.000000
50%,0.662000,0.651000,5.000000,-5.732000,1.000000,0.051850,0.128500,0.000000,0.127500,0.442000,121.288000,189520.000000,4.000000
75%,0.754000,0.754000,9.000000,-4.691500,1.000000,0.080525,0.355750,0.000021,0.197500,0.672750,139.284750,218551.000000,4.000000
max,0.943000,0.887000,11.000000,-2.248000,1.000000,0.331000,0.959000,0.598000,0.607000,0.962000,203.759000,277832.000000,4.000000


In [30]:
hot_songs_with_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Song_title        84 non-null     object 
 1   Artist            84 non-null     object 
 2   id                84 non-null     object 
 3   danceability      84 non-null     float64
 4   energy            84 non-null     float64
 5   key               84 non-null     int64  
 6   loudness          84 non-null     float64
 7   mode              84 non-null     int64  
 8   speechiness       84 non-null     float64
 9   acousticness      84 non-null     float64
 10  instrumentalness  84 non-null     float64
 11  liveness          84 non-null     float64
 12  valence           84 non-null     float64
 13  tempo             84 non-null     float64
 14  type              84 non-null     object 
 15  id                84 non-null     object 
 16  uri               84 non-null     object 
 17 

In [31]:
#saving new dataframe to csv
hot_songs_with_features.to_csv('hot_songs_with_features.csv', index=False)

In [46]:
hot_songs_with_features.columns

Index(['track_name', 'artist_name', 'id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href',
       'analysis_url', 'duration_ms', 'time_signature'],
      dtype='object')

In [45]:
hot_songs_with_features.rename(columns={'Song_title': 'track_name', 'Artist': 'artist_name'}, inplace=True)

### add not hot songs csv

In [29]:
#add csv to pandas
not_hot_songs = pd.read_csv('not_so_hot_songs_with_audio_features.csv')

In [30]:
not_hot_songs.columns

Index(['Song_title', 'Artist', 'genre', 'release_date', 'topic', 'len', 'id',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id.1', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [ ]:
# 7 replace the old internal files with the ones including audio features

In [ ]:
# 8 create functions file

In [44]:
#WIP

In [50]:
def remove_non_common_columns(df1:pd.DataFrame, df2:pd.DataFrame) -> pd.DataFrame:
    '''
    This function removes non-common columns from two dataframes. 
    It first finds the common columns between the two dataframes, 
    then creates copies of the original dataframes and selects only the common columns. 
    The modified dataframes are then returned.
    '''
    common_cols = list(set(df1.columns).intersection(set(df2.columns)))
    df1c=df1.copy()
    df2c=df2.copy()
    df1c = df1c[common_cols]
    df2c = df2c[common_cols]
    return df1c, df2c

In [52]:
dataframe_01, dataframe_02 =remove_non_common_columns(hot_songs_with_features, not_hot_songs)

In [31]:
dataframe_01.shape

NameError: name 'dataframe_01' is not defined

In [55]:
dataframe_02.columns

Index(['energy', 'danceability', 'instrumentalness', 'artist_name', 'valence',
       'track_name', 'loudness', 'acousticness'],
      dtype='object')

In [58]:
cols = dataframe_01.columns.tolist()
cols = ['artist_name', 'track_name', 'energy',
 'danceability',
 'instrumentalness',
 'valence',
 'loudness',
 'acousticness']

In [61]:
hot_songs_clean=dataframe_01[cols]

In [63]:
not_hot_songs_clean=dataframe_02[cols]

In [64]:
#saving new dataframe to csv
hot_songs_clean.to_csv('hot_songs_clean.csv', index=False)

In [65]:
#saving new dataframe to csv
not_hot_songs_clean.to_csv('not_hot_songs_clean.csv', index=False)